In [ ]:
from ast import literal_eval
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix, load_npz, save_npz
from tqdm import tqdm
from sklearn.preprocessing import normalize

import seaborn as sns
import datetime
import json
import numpy as np
import pandas as pd
import time
import yaml
import scipy.sparse as sparse
from ast import literal_eval

# For Python2 this have to be done
# from __future__ import division


### Import Toronto data as df,  userJson as df_user

In [29]:
df = pd.read_csv('../data/toronto_reviews.csv')

In [ ]:
reviewJson = '../data/Export_CleanedReview.json'
# file directory
reviewJson, file_user = [
    '..\\data\\Export_CleanedReview.json',
    '..\\data\\user.json',
]

In [48]:
# with open(reviewJson,'r') as f:
#     data = f.readlines()
#     data = list(map(json.loads, data))
# df_review = pd.DataFrame(data[0])
# df_review

,business_id,cool,date,funny,review_id,stars,text,useful,user_avg_stars,user_id
0,--1UhMGODdWsrMastO9DZw,0,2016-08-16,0,8UsalYi1Jkbk8WitFEa_aQ,4,Spicy Amigos is the old Jalapeños. I'm not sur...,0,3.14,1b4rlFFdNfeysOjbrFykng
1,--1UhMGODdWsrMastO9DZw,0,2017-07-26,0,X4PwY0mMHxqOf2O1pwYdIQ,5,"Delicious! One of the best burritos, salsa and...",0,3.92,4YHZzwlxEMi7zWO6Osszdw
2,--1UhMGODdWsrMastO9DZw,0,2016-06-13,0,C_pdS4RGgrPrt4ctYOrQ0A,3,One of the better down to earth Tacos I have h...,0,4.25,A4bpHuvzaQt9-XAg8e9Msw
3,--1UhMGODdWsrMastO9DZw,1,2016-11-14,0,aIzkfP0-30mpNcBo9MsjmA,4,Great Mexican joint in the west side of downto...,1,3.53,BgTWMo2qRrXINPiM35w8zw
4,--1UhMGODdWsrMastO9DZw,0,2018-05-06,0,8QUwzeXeyJ3L15lKfhKLsQ,1,I had the chicken Tinga enchilada in mole sauc...,0,2.79,Bsy9F-59sl9OT_bvZNl3hA
5,--1UhMGODdWsrMastO9DZw,0,2018-01-11,0,hhVr1uH7XaRlbIHI8dYvbA,5,Glad these guys moved in on the block! It's qu...,0,3.75,FYhU1fKQ7n11WQ7gcFYOag
6,--1UhMGODdWsrMastO9DZw,2,2016-06-18,0,GSTQtPuRgIGveAAJYaIn3A,5,"Great food, service and price. Everything we t...",1,3.26,GL81ktDIteXA2VVH6gIakg
7,--1UhMGODdWsrMastO9DZw,0,2017-04-03,0,Ueb33Y77jRDQhkjf7nmCUw,5,"We were in the mood for tacos, and came across...",0,4.17,Le81nQXs_LwAxRk9eK8oQQ
8,--1UhMGODdWsrMastO9DZw,0,2017-08-09,0,oKm8UTv-QSC0oCbniqwxjg,4,"Tasty, authentic Mexican street food that give...",0,4.63,NoQCmYKyMPs4D01Wa6dZew
9,--1UhMGODdWsrMastO9DZw,2,2016-06-04,0,Mc0ecKvOZmDGPDt1UdQcBw,1,"Greasy, tasteless meat, only used one taco ins...",5,2.73,NqpKiaRsGfuU2voV5dPRCQ


In [ ]:
with open(file_user, encoding = 'utf-8') as json_file:
    data = json_file.readlines()
    data = list(map(json.loads, data))

df_user = pd.DataFrame(data)

### Data Processing

In [30]:
# Rename Month as Day and Day as Month(error occurs on original data)
df=df.rename(columns = {'Month':'new_Day'})
df=df.rename(columns = {'Day':'Month'})
df=df.rename(columns = {'new_Day':'Day'})

In [33]:
# Merge the Year, Month Day as to one column
df['date'] = pd.to_datetime(df[['Year', 'Month','Day']])

In [34]:
# df.columns.values
df

,Unnamed: 0,Unnamed: 0.1,business_id,friend_count,ghost,img_dsc,img_url,nr,photo_count,rating,...,review_text,ufc,user_id,user_loc,vote_count,Updated,Year,Day,Month,date
0,0,0,bSQGCheX1BwvL67Az1OJlA,90,False,[],[],False,231.0,5.0,...,I had a good experience at this store. I was g...,"[10, 1, 10]",q41qqkChL9fRxF43cnkgbw,"Houston, TX",11.0,False,2018,5,9,2018-09-05
1,1,1,bSQGCheX1BwvL67Az1OJlA,0,False,[],[],False,9.0,1.0,...,I reaaaaaally wish I had read all the stories ...,"[0, 0, 0]",tNT5I9-QcljS-7w_doXCXQ,"Toronto, Canada",0.0,False,2018,6,3,2018-03-06
2,2,2,bSQGCheX1BwvL67Az1OJlA,52,False,"[""Photo of Sleep Country - Toronto, ON, Canada...",['https://s3-media1.fl.yelpcdn.com/bphoto/E_gL...,False,27.0,1.0,...,Everything was fine until the delivery. I wait...,"[3, 1, 2]",9q1Do6TVM27RNHUQAQxPRg,"Toronto, Canada",3.0,False,2016,30,7,2016-07-30
3,3,3,Z7JV11GXw_RsGr8Ik_Hsag,0,False,[],[],False,NaN,5.0,...,"Thought I had bed bugs, but it turned out to b...","[0, 0, 0]",cr-4gAf1_seYWLoO59Lp4w,"Old Toronto, Toronto, Canada",0.0,False,2018,16,12,2018-12-16
4,4,4,Z7JV11GXw_RsGr8Ik_Hsag,0,False,[],[],False,NaN,5.0,...,"When we called, Mr. Akram was willing to come ...","[1, 0, 0]",CQkTIyhNczRlRI4kIdAzSw,"Toronto, Canada",1.0,False,2014,6,8,2014-08-06
5,5,5,kI7ObOAe-Wvjdtnv-8Cu7w,74,False,[],[],False,NaN,5.0,...,Great chapter! The education courses are caree...,"[0, 0, 0]",lhiHff4DQ8aleZeTzmC3TA,"Etobicoke, Canada",0.0,False,2015,16,8,2015-08-16
6,6,6,Xo1LNzhnwE-ilqsM3ybs9Q,105,False,"['Photo of Happy Lemon - Markham, ON, Canada. ...",['https://s3-media4.fl.yelpcdn.com/bphoto/tu7j...,False,325.0,4.0,...,I ordered the lemon mango slush and the lemon ...,"[1, 1, 1]",zsJFjhBQEFQ6gJ7BsNM_Ug,"Toronto, Canada",1.0,False,2016,23,8,2016-08-23
7,7,7,Xo1LNzhnwE-ilqsM3ybs9Q,171,False,"['Photo of Happy Lemon - Markham, ON, Canada. ...",['https://s3-media3.fl.yelpcdn.com/bphoto/h110...,False,307.0,3.0,...,"Came here on a Sunday afternoon, it wasn't bus...","[1, 0, 0]",P7YuMh74-I2cDq7oU8frww,"York Regional Municipality, Canada",1.0,False,2016,2,10,2016-10-02
8,8,8,Xo1LNzhnwE-ilqsM3ybs9Q,239,False,"['Photo of Happy Lemon - Markham, ON, Canada. ...",['https://s3-media4.fl.yelpcdn.com/bphoto/tS6Y...,False,78.0,5.0,...,{Grapefruit & Yakult Green Tea with Aloe Jelly...,"[0, 0, 0]",C91it8b3odRg0503asUaJA,"Markham, Canada",0.0,False,2016,6,11,2016-11-06
9,9,9,Xo1LNzhnwE-ilqsM3ybs9Q,10,False,"['Photo of Happy Lemon - Markham, ON, Canada',...",['https://s3-media3.fl.yelpcdn.com/bphoto/rfB0...,False,504.0,4.0,...,Saw this newly opened bubble tea shop and want...,"[0, 0, 0]",dAB-v4B-5_8QgTcoVSnZOw,"Vaughan, Canada",0.0,False,2016,25,9,2016-09-25


In [35]:
# Drop the unuseful columns
df.drop(columns=['Year', 'Month','Day','Unnamed: 0','Unnamed: 0.1','friend_count','ghost','img_dsc','img_url','nr','photo_count','review_count','review_date','review_language','ufc','user_loc','vote_count','Updated'],inplace=True)

In [36]:
# Inner join the df with df_user to get the average_stars column
df = pd.merge(df, df_user[['average_stars','user_id']], on='user_id', how='inner')

In [37]:
df

,business_id,rating,review_id,review_text,user_id,date,average_stars
0,bSQGCheX1BwvL67Az1OJlA,5.0,NjYpdf2W7_2UU691w9Rlnw,I had a good experience at this store. I was g...,q41qqkChL9fRxF43cnkgbw,2018-09-05,4.29
1,DFFiZZyCjciD_j1wgM3N0w,5.0,KlzAeONIVlEe43TqsGUZXw,"Great, quick, and easy experience with Sheldon...",q41qqkChL9fRxF43cnkgbw,2017-12-30,4.29
2,j7_yOwPVGPrzsF6uNOngEg,4.0,XfoUEjgn7fXDXkmyfaqJwA,"Pretty small store and selection is limited, b...",q41qqkChL9fRxF43cnkgbw,2017-11-08,4.29
3,_4B99coJlSasAgQPm0gyqA,5.0,rXKs_11BT3s7KiQ2rlcPaA,It was our first time using Wild North Flowers...,q41qqkChL9fRxF43cnkgbw,2017-12-24,4.29
4,W2W3tTQkid1Z5PXTnVQP6Q,5.0,YMotxam_oSk6uFEPkVYvLg,I have been to Reptilia with both adults and l...,q41qqkChL9fRxF43cnkgbw,2018-08-24,4.29
5,_cjPEH9wXhKS-HQe_U3M4Q,3.0,s2AuZqdXrND_LtMh7_YeZA,This was maybe my fourth or fifth time at this...,q41qqkChL9fRxF43cnkgbw,2018-08-11,4.29
6,RA00GtlU25TkIP9i9wd8bg,3.0,ilbELhGN4eQw9PYOO7gFFg,Super pricey. While some of the exhibits were ...,q41qqkChL9fRxF43cnkgbw,2017-11-22,4.29
7,gnCl7VXV5BALM0SqOHHmGg,5.0,0YRlmlpv_K-_Bt8VBl1E2w,We followed Dr. Kulik from her previous office...,q41qqkChL9fRxF43cnkgbw,2018-08-17,4.29
8,Tga64GlSy6xjp-Z-Ogddng,4.0,_06CaWLnmbo9P9C2NckuQQ,We have been customers of crate and barrel for...,q41qqkChL9fRxF43cnkgbw,2018-08-25,4.29
9,WkdIxfgIj-iH1OL4YJ6oFQ,5.0,EGR9C15IhlPkrPjXYZG9yA,Super simple process. I was looking for a rela...,q41qqkChL9fRxF43cnkgbw,2018-08-25,4.29


In [39]:
# Rename all the fields
df.rename(columns={"rating": "stars","review_text":"text","average_stars":"user_avg_stars"},inplace = True)

In [69]:
df.date = df.date.astype("str")

In [72]:
Export = df.to_json(r'..\\data\\Export_TorontoData.json', orient='records')